<a href="https://colab.research.google.com/github/SlowPoke228/Lessons/blob/main/L2(Analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import pickle
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


from google.colab import drive
drive.mount('/content/drive')
# url db: https://www.kaggle.com/christianlillelund/csgo-round-winner-classification
# url lesson: https://colab.research.google.com/drive/1qUDkan8KSUYSauxNzPUZEZi93LlT7l3E
# Указываем тайтлы для столбцов
names = ['time_left','ct_score','t_score','map','bomb_planted','ct_health','t_health','ct_armor','t_armor','ct_money','t_money','ct_helmets',
        't_helmets','ct_defuse_kits','ct_players_alive','t_players_alive','ct_weapon_ak47','t_weapon_ak47','ct_weapon_aug','t_weapon_aug','ct_weapon_awp'
        ,'t_weapon_awp','ct_weapon_bizon','t_weapon_bizon','ct_weapon_cz75auto','t_weapon_cz75auto','ct_weapon_elite','t_weapon_elite','ct_weapon_famas',
        't_weapon_famas','ct_weapon_g3sg1','t_weapon_g3sg1','ct_weapon_galilar','t_weapon_galilar','ct_weapon_glock','t_weapon_glock','ct_weapon_m249','t_weapon_m249'
        ,'ct_weapon_m4a1s','t_weapon_m4a1s','ct_weapon_m4a4','t_weapon_m4a4','ct_weapon_mac10','t_weapon_mac10','ct_weapon_mag7'
         ,'t_weapon_mag7','ct_weapon_mp5sd','t_weapon_mp5sd','ct_weapon_mp7','t_weapon_mp7','ct_weapon_mp9','t_weapon_mp9','ct_weapon_negev','t_weapon_negev','ct_weapon_nova',
        't_weapon_nova','ct_weapon_p90','t_weapon_p90','ct_weapon_r8revolver','t_weapon_r8revolver','ct_weapon_sawedoff','t_weapon_sawedoff','ct_weapon_scar20',
       't_weapon_scar20','ct_weapon_sg553','t_weapon_sg553','ct_weapon_ssg08','t_weapon_ssg08','ct_weapon_ump45','t_weapon_ump45','ct_weapon_xm1014',
       't_weapon_xm1014','ct_weapon_deagle','t_weapon_deagle','ct_weapon_fiveseven','t_weapon_fiveseven','ct_weapon_usps','t_weapon_usps','ct_weapon_p250','t_weapon_p250',
       'ct_weapon_p2000','t_weapon_p2000','ct_weapon_tec9','t_weapon_tec9','ct_grenade_hegrenade','t_grenade_hegrenade','ct_grenade_flashbang','t_grenade_flashbang',
       'ct_grenade_smokegrenade','t_grenade_smokegrenade','ct_grenade_incendiarygrenade','t_grenade_incendiarygrenade','ct_grenade_molotovgrenade','t_grenade_molotovgrenade',
       'ct_grenade_decoygrenade','t_grenade_decoygrenade','round_winner'
]

# Подключаем бд, удаляем первую стоку и указываем тайтлы
dataset = pd.read_csv("/content/drive/MyDrive/csgo_round_snapshots.csv", names=names, skiprows=1)
dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,t_weapon_aug,ct_weapon_awp,t_weapon_awp,ct_weapon_bizon,t_weapon_bizon,ct_weapon_cz75auto,t_weapon_cz75auto,ct_weapon_elite,t_weapon_elite,ct_weapon_famas,t_weapon_famas,ct_weapon_g3sg1,t_weapon_g3sg1,ct_weapon_galilar,t_weapon_galilar,ct_weapon_glock,t_weapon_glock,ct_weapon_m249,t_weapon_m249,ct_weapon_m4a1s,t_weapon_m4a1s,...,t_weapon_p90,ct_weapon_r8revolver,t_weapon_r8revolver,ct_weapon_sawedoff,t_weapon_sawedoff,ct_weapon_scar20,t_weapon_scar20,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,t_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_xm1014,t_weapon_xm1014,ct_weapon_deagle,t_weapon_deagle,ct_weapon_fiveseven,t_weapon_fiveseven,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_weapon_p2000,t_weapon_p2000,ct_weapon_tec9,t_weapon_tec9,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,de_train,True,200.0,242.0,195.0,359.0,100.0,5950.0,2.0,4.0,1.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,T
122406,174.93,11.0,15.0,de_train,False,500.0,500.0,95.0,175.0,11500.0,23900.0,1.0,2.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
#удаляем данные с слишком высокой и слишком низкой корреляцией
dataset = dataset.drop(['map','bomb_planted','ct_weapon_bizon','t_weapon_bizon','ct_weapon_cz75auto','t_weapon_cz75auto','ct_weapon_elite','t_weapon_elite','ct_weapon_famas',
        't_weapon_famas','ct_weapon_g3sg1','t_weapon_g3sg1','ct_weapon_galilar','t_weapon_galilar','ct_weapon_glock','ct_weapon_m249','t_weapon_m249'
        ,'ct_weapon_m4a1s','t_weapon_m4a1s','ct_weapon_mac10','ct_weapon_mag7','t_weapon_mag7','ct_weapon_mp5sd','t_weapon_mp5sd','ct_weapon_mp7','t_weapon_mp7','t_weapon_mp9',
        'ct_weapon_negev','t_weapon_negev','ct_weapon_nova','ct_weapon_xm1014','t_weapon_xm1014','t_weapon_nova','ct_weapon_p90','t_weapon_p90','ct_weapon_r8revolver','t_weapon_r8revolver',
        'ct_weapon_sawedoff','t_weapon_sawedoff','ct_weapon_scar20','t_weapon_scar20','ct_grenade_decoygrenade','t_grenade_decoygrenade','t_grenade_incendiarygrenade','ct_weapon_p2000','t_weapon_p2000',
        'ct_weapon_tec9','t_weapon_tec9','t_weapon_aug','ct_players_alive','t_players_alive','t_weapon_m4a4','t_weapon_ssg08','ct_grenade_molotovgrenade','ct_weapon_fiveseven','t_weapon_fiveseven'], 
        axis='columns')
# Создаем массивы с атрибутами и метками
X = dataset.iloc[:, :-1].values    
y = dataset.iloc[:, len(dataset.columns)-1].values
# разделяем массивы на тренеровочные и тестовые 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10,random_state=42,stratify=y)
print("Качество алгоритма до нормализации:", KNeighborsClassifier(n_neighbors=1).fit(X_train, y_train).score(X_test, y_test))

Качество алгоритма до нормализации: 0.8221550526917736


In [ ]:
#scaler = StandardScaler()
scaler = MinMaxScaler()
out=[]
output=open('data.pickle', 'wb')
for clf in [DecisionTreeClassifier(), SGDClassifier(),  GaussianNB(),KNeighborsClassifier(n_neighbors=1) ]:  #SVC(),
  pipe = Pipeline([('normalizer', scaler), ('classifier', clf)],verbose=True)
  pipe.fit(X_train,y_train)
  cfm=confusion_matrix(y_test, pipe.predict(X_test))
  clfr=classification_report(y_test, pipe.predict(X_test))
  out.append([clf.__class__.__name__,cfm,clfr])
  print("Качество алгоритма после нормализации:",
        clf.__class__.__name__, 
        cfm,
        clfr, sep="\n"
        )
pickle.dump(out, output)
output.close()


[Pipeline] ........ (step 1 of 2) Processing normalizer, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   2.3s
Качество алгоритма после нормализации:
DecisionTreeClassifier
[[4916 1084]
 [1107 5134]]
              precision    recall  f1-score   support

          CT       0.82      0.82      0.82      6000
           T       0.83      0.82      0.82      6241

    accuracy                           0.82     12241
   macro avg       0.82      0.82      0.82     12241
weighted avg       0.82      0.82      0.82     12241

[Pipeline] ........ (step 1 of 2) Processing normalizer, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   0.6s
Качество алгоритма после нормализации:
SGDClassifier
[[5226  774]
 [2401 3840]]
              precision    recall  f1-score   support

          CT       0.69      0.87      0.77      6000
           T       0.83      0.62      0.71      6241

    accuracy                           0.74     12241
  

In [ ]:
with open('data.pickle', 'rb') as f:
  data_new = pickle.load(f)
  for i in data_new:
    for item in i:
      print(item)

DecisionTreeClassifier
[[4916 1084]
 [1107 5134]]
              precision    recall  f1-score   support

          CT       0.82      0.82      0.82      6000
           T       0.83      0.82      0.82      6241

    accuracy                           0.82     12241
   macro avg       0.82      0.82      0.82     12241
weighted avg       0.82      0.82      0.82     12241

SGDClassifier
[[5226  774]
 [2401 3840]]
              precision    recall  f1-score   support

          CT       0.69      0.87      0.77      6000
           T       0.83      0.62      0.71      6241

    accuracy                           0.74     12241
   macro avg       0.76      0.74      0.74     12241
weighted avg       0.76      0.74      0.74     12241

GaussianNB
[[4299 1701]
 [1796 4445]]
              precision    recall  f1-score   support

          CT       0.71      0.72      0.71      6000
           T       0.72      0.71      0.72      6241

    accuracy                           0.71     12241